In [210]:
import pandas as pd
import numpy as np
from itertools import combinations, chain
from prettytable import PrettyTable

In [211]:
min_support = int(input("Please Enter Support in %: "))
min_confidence = int(input("Please Enter Confidence in %: "))

Please Enter Support in %: 30
Please Enter Confidence in %: 30


In [212]:
menubar = {}
menubar["1"] = "Amazon"
menubar["2"] = "Best Buy"
menubar["3"] = "Nike"
menubar["4"] = "K-mart"
menubar["5"] = "Generic"
 
options=menubar.keys()
while True:  
    for entry in options: 
        print(entry, menubar[entry])
    selected_opt=input("What Database do you want to select?[1-5] ") 
    if selected_opt=="1": 
        dataset="Amazon_Item_Names.xlsx" 
        transactions="Amazon_Datasets.xlsx"
        print("Amazon is selected.")
        break
    elif selected_opt=="2":
        dataset="Bestbuy_item_names.xlsx" 
        transactions="Bestbuy_datasets.xlsx"
        print("Best Buy is selected.")
        break
    elif selected_opt=="3":
        dataset="Nike_item_names.xlsx" 
        transactions="Nike_datasets.xlsx"
        print("Nike is selected.")
        break
    elif selected_opt=="4":
        dataset="Kmart_item_names.xlsx" 
        transactions="Kmart_datasets.xlsx"
        print("K-mart is selected.")
        break
    elif selected_opt=="5":
        dataset="Generic_item_names.xlsx" 
        transactions="Generic_datasets.xlsx"
        print("Generic is selected.")
        break
    
    elif selected_opt !="":
      selected_opt = input("\n Not Valid Choice Try again")

1 Amazon
2 Best Buy
3 Nike
4 K-mart
5 Generic
What Database do you want to select?[1-5] 5
Generic is selected.


In [213]:
items_list = pd.read_excel(dataset,names=["item#", "items"]) 
items=[] 
for item in items_list["items"]:
    items.append(item)  
print(items)

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']


In [214]:
transaction_list = pd.read_excel(transactions,names=["Transaction ID","Transaction"])
transaction_list.head()

,Transaction ID,Transaction
0,Trans1,"A, B, C"
1,Trans2,"A, B, C"
2,Trans3,"A, B, C, D"
3,Trans4,"A, B, C, D, E"
4,Trans5,"A, B, D, E"


In [215]:
records=[]
for  trans in transaction_list["Transaction"]:
    transactions =  trans.split(", ")
    records.append(transactions)
print((records))

[['A', 'B', 'C'], ['A', 'B', 'C'], ['A', 'B', 'C', 'D'], ['A', 'B', 'C', 'D', 'E'], ['A', 'B', 'D', 'E'], ['A', 'D', 'E'], ['A', 'E'], ['A', 'E'], ['A', 'C', 'E'], ['A', 'C', 'E'], ['A', 'C', 'E'], ['B,D,F,G'], ['H,I,J'], ['A,D,E,H,J'], ['B,C,E,F'], ['A,D,F'], ['B,D,J'], ['A,B,C,J'], ['B,D,E,G'], ['A,C,E,H,J']]


In [216]:
C={} 
L = {} 
itemset_size=1 
Discarded= {itemset_size: []} 
C.update({itemset_size: [ [f] for f in items]})

In [217]:
def count_occurances(itemset,records):
    count = 0
    for i in range(len(records)):
        if set(itemset).issubset(set(records[i])):  
                                                   
            count+=1
          
    return count

In [218]:
def join_two_itemsets(it1,it2,items):
    it1.sort(key=lambda x: items.index(x)) 
    it2.sort(key=lambda x: items.index(x))
    for i in range(len(it1)-1):
        if it1[i] != it2[i]:  
            return []
    if items.index(it1[-1])< items.index(it2[-1]): 
        return it1 + [it2[-1]]
    return []

In [219]:
def join_set_itemsets(set_of_itemsets,items):
    C=[]
    for i in range(len(set_of_itemsets)):
        for j in range(i+1,len(set_of_itemsets)):
            it_out= join_two_itemsets(set_of_itemsets[i],set_of_itemsets[j],items)
            if len(it_out) >0 :
                C.append(it_out)
            
    return C

In [220]:
def get_frequent_items(itemsets,records, min_support, prev_non_frequent_items):
    L = [] 
    supp_count = []
    new_non_frequent_items =[] 
    k= len(prev_non_frequent_items.keys())
    for i in range(len(itemsets)):
        non_frequent_items_before = False
        if k>0:
            for  item in prev_non_frequent_items[k]:
                if set(item).issubset(set(itemsets[i])):
                    non_frequent_items_before = True
                    break
        if not non_frequent_items_before:
            count = count_occurances(itemsets[i], records)
            if (100*(count/20)) >= min_support:
                L.append(itemsets[i])
                supp_count.append(count)
        else:
            new_discarded.append(itemsets[i])
   
    return L, supp_count, new_non_frequent_items

In [221]:
def print_table(list_of_itemsets,supp_count):
    t = PrettyTable(['Itemset', 'Frequency', 'Support (%)'])
    for k in range(len(list_of_itemsets)):
        t.add_row([list_of_itemsets[k] , supp_count[k], round(((supp_count[k]/20)*100),2)])
    print(t)

In [222]:
def create_set_combinations(s):
    return list(chain.from_iterable(combinations(s,r) for r in range(1, len(s)+1)))

In [223]:
def generate_rules(X,X_S,S,conf,supp_x):
    rule = ""
    rule += "Freq. Itemset: {}\n".format(X)
    rule += "      Rule: {} -> {} \n".format(list(S),list(X_S))
    rule += "      Confidence: {}% ".format(conf)
    rule += "      Support: {}% \n\n".format(supp_x)
    rule+= "----------------------------------------------------------------------------------------------------------\n"
    return rule

In [224]:
support_count_L ={} 
freq_items, sup, new_non_frequent_items = get_frequent_items(C[itemset_size], records, min_support, Discarded)
Discarded.update({itemset_size:new_non_frequent_items}) 
L.update({itemset_size:freq_items})
support_count_L.update({itemset_size:sup})
print(L)
print(support_count_L)

{1: [['A'], ['C'], ['E']]}
{1: [11, 7, 8]}


In [225]:
print("Table C1: ")
print_table(C[1],[count_occurances(it,records) for it in C[1]]) 
print("Table L1 : ")
print_table(L[1],support_count_L[1]) 

Table C1: 
+---------+-----------+-------------+
| Itemset | Frequency | Support (%) |
+---------+-----------+-------------+
|  ['A']  |     11    |     55.0    |
|  ['B']  |     5     |     25.0    |
|  ['C']  |     7     |     35.0    |
|  ['D']  |     4     |     20.0    |
|  ['E']  |     8     |     40.0    |
|  ['F']  |     0     |     0.0     |
|  ['G']  |     0     |     0.0     |
|  ['H']  |     0     |     0.0     |
|  ['I']  |     0     |     0.0     |
|  ['J']  |     0     |     0.0     |
+---------+-----------+-------------+
Table L1 : 
+---------+-----------+-------------+
| Itemset | Frequency | Support (%) |
+---------+-----------+-------------+
|  ['A']  |     11    |     55.0    |
|  ['C']  |     7     |     35.0    |
|  ['E']  |     8     |     40.0    |
+---------+-----------+-------------+


In [226]:
k = itemset_size + 1 
convergence = False 

while not convergence:
    C.update({k: join_set_itemsets(L[k-1],items)})
    print("Table C{}".format(k))
    print_table(C[k],[count_occurances(it,records) for it in C[k]])
    f, supp, new_discarded = get_frequent_items(C[k], records, min_support, Discarded)
    Discarded.update({k: new_discarded})
    L.update({k: f})
    support_count_L.update({k: supp})
    if len(L[k]) == 0:
        convergence = True
    print("Table L{}".format(k))
    print_table(L[k],support_count_L[k])
    k+=1

Table C2
+------------+-----------+-------------+
|  Itemset   | Frequency | Support (%) |
+------------+-----------+-------------+
| ['A', 'C'] |     7     |     35.0    |
| ['A', 'E'] |     8     |     40.0    |
| ['C', 'E'] |     4     |     20.0    |
+------------+-----------+-------------+
Table L2
+------------+-----------+-------------+
|  Itemset   | Frequency | Support (%) |
+------------+-----------+-------------+
| ['A', 'C'] |     7     |     35.0    |
| ['A', 'E'] |     8     |     40.0    |
+------------+-----------+-------------+
Table C3
+-----------------+-----------+-------------+
|     Itemset     | Frequency | Support (%) |
+-----------------+-----------+-------------+
| ['A', 'C', 'E'] |     4     |     20.0    |
+-----------------+-----------+-------------+
Table L3
+---------+-----------+-------------+
| Itemset | Frequency | Support (%) |
+---------+-----------+-------------+
+---------+-----------+-------------+


In [227]:
association_rules="" 
for i in range(1,len(L)):
    for j in range(len(L[i])):
        sets_list = create_set_combinations(set(L[i][j]))
        sets_list.pop() 
        for z in sets_list:
            S=set(z) 
            X= set(L[i][j])
            X_S = set(X-S) 
            count_x = count_occurances(X, records)
            supp_x = round(((count_x/20)*100),2)
            count_x_s = count_occurances(X_S, records)
            supp_x_s = count_occurances(X_S, records)
            count_s = count_occurances(S,records)
            supp_s = round(((count_s/20)*100),2)
            
            conf =  round(((supp_x/supp_s)*100),2)
            if conf > min_confidence and supp_x > min_support:
                association_rules+= generate_rules(X,X_S,S,conf,supp_x)

In [228]:
print(association_rules)

Freq. Itemset: {'C', 'A'}
      Rule: ['C'] -> ['A'] 
      Confidence: 100.0%       Support: 35.0% 

----------------------------------------------------------------------------------------------------------
Freq. Itemset: {'C', 'A'}
      Rule: ['A'] -> ['C'] 
      Confidence: 63.64%       Support: 35.0% 

----------------------------------------------------------------------------------------------------------
Freq. Itemset: {'E', 'A'}
      Rule: ['E'] -> ['A'] 
      Confidence: 100.0%       Support: 40.0% 

----------------------------------------------------------------------------------------------------------
Freq. Itemset: {'E', 'A'}
      Rule: ['A'] -> ['E'] 
      Confidence: 72.73%       Support: 40.0% 

----------------------------------------------------------------------------------------------------------

